In [30]:
from src.get_modelnet40.load_data import get_dls_for_viz
from src.it_net.it_net import ITNet
import torch

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pyvista as pv
from tqdm import tqdm
import random

import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
from src.pretrain_utils.transforms import quaternion_to_matrix, apply_tfm, compose_tfms
from src.pretrain_utils.corruptions import tfm_from_rand_pose, create_random_transform

In [5]:
device = "mps"
batch_size = 64

In [6]:
checkpoint = torch.load("results/it_net2/checkpoint.pth")

model = ITNet(channel=3, num_iters=5).to(device)
model.load_state_dict(checkpoint["model"])
model = model.eval()

In [7]:
train_loader = get_dls_for_viz(batch_size=batch_size)
batch = next(iter(train_loader))

In [8]:
# batch_point_clouds = batch["pointcloud"].to(torch.float32).to(device)
# batch_point_clouds = batch_point_clouds.transpose(1, 2)

In [19]:
batch_point_clouds = batch["pointcloud"].float().to(device)
batch_point_clouds = batch_point_clouds.transpose(1, 2)

In [24]:
tfm = create_random_transform(batch_size, 30, 0, batch_point_clouds.dtype).to(batch_point_clouds.device)
view_1 = apply_tfm(batch_point_clouds, tfm)
view_1_post, _, _ = model(view_1)


tfm = create_random_transform(batch_size, 30, 0, view_1.dtype).to(view_1.device)
view_2 = apply_tfm(view_1, tfm)
view_2_post, _, _ = model(view_2)

# Visualize

In [25]:
min_xyz = [-1,-1,-1]
max_xyz = [1,1,1]

In [31]:
def pcshow(xs_list, ys_list, zs_list, min_xyz, max_xyz):
    # Create subplots with 1 row and 2 columns
    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]])

    # Iterate over the provided point clouds
    for i, (xs, ys, zs) in enumerate(zip(xs_list, ys_list, zs_list), start=1):
        # Add Scatter3d trace for each point cloud to respective subplot
        fig.add_trace(go.Scatter3d(x=xs, y=ys, z=zs, mode='markers', 
                                    name=f'Point Cloud {i}', marker=dict(size=2)), row=1, col=i)

    # Update layout with aspect ratio mode set to 'cube'
    fig.update_layout(scene=dict(aspectmode='cube'))

    # Show the figure
    fig.show()

In [32]:
indices = list()
wanted_category = 0

for index in range(0, batch_size):
    category = batch["category"][index]
    if category == wanted_category:
        indices.append(index)

index = random.choice(indices)

In [33]:
cloud1 = view_1[index,:,:].transpose(0, 1).detach().cpu().numpy()
x1 = cloud1[:,0]
y1 = cloud1[:,1]
z1 = cloud1[:,2]

cloud2 = view_1_post[index,:,:].transpose(0, 1).detach().cpu().numpy()
x2 = cloud2[:,0]
y2 = cloud2[:,1]
z2 = cloud2[:,2]


# Example usage with two point clouds
pcshow([x1, x2], [y1, y2], [z1, z2], min_xyz, max_xyz)

In [34]:
cloud1 = view_2[index,:,:].transpose(0, 1).detach().cpu().numpy()
x1 = cloud1[:,0]
y1 = cloud1[:,1]
z1 = cloud1[:,2]

cloud2 = view_2_post[index,:,:].transpose(0, 1).detach().cpu().numpy()
x2 = cloud2[:,0]
y2 = cloud2[:,1]
z2 = cloud2[:,2]


# Example usage with two point clouds
pcshow([x1, x2], [y1, y2], [z1, z2], min_xyz, max_xyz)